
## **Chapter 1: Introduction to Time-Series Prediction Systems**

---

### **1.1 What is a Time-Series Prediction System?**

A time-series prediction system is a specialized computational framework designed to analyze historical data collected over time and generate forecasts about future values. Unlike traditional machine learning problems where data points are assumed to be independent, time-series data has an inherent temporal ordering where each observation is related to its predecessors and successors.

#### **Understanding the Core Concept**

At its heart, a time-series prediction system answers a fundamental question: *"Given what has happened in the past, what is likely to happen in the future?"* This question manifests differently across domains:

- **Finance**: "Given the historical stock prices, what will the price be tomorrow?"
- **Retail**: "Given past sales data, how much inventory should we stock next month?"
- **Weather**: "Given temperature readings over the past week, what will the temperature be this weekend?"
- **Healthcare**: "Given a patient's vital signs over time, will their condition deteriorate?"

#### **The NEPSE Example Context**

Let's ground this concept with our running example throughout this handbook: the **Nepal Stock Exchange (NEPSE)** prediction system. NEPSE is the primary stock exchange of Nepal, and predicting its movements involves analyzing historical trading data to forecast future price movements.

Consider a sample from our NEPSE dataset:

```csv
S.No,Symbol,Conf.,Open,High,Low,Close,LTP,Close - LTP,Close - LTP %,VWAP,Vol,Prev. Close,Turnover,Trans.,Diff,Range,Diff %,Range %,VWAP %,52 Weeks High,52 Weeks Low
1,ABL,Standard,580.00,590.00,575.00,588.00,588.00,0.00,0.00,584.50,15230,580.00,8900000,285,8.00,15.00,1.38,2.59,0.77,650.00,420.00
2,ADBL,Standard,620.00,635.00,615.00,630.00,630.00,0.00,0.00,626.80,8500,618.00,5330000,152,12.00,20.00,1.94,3.24,1.42,720.00,450.00
```

Each row represents a single trading day for a specific stock symbol. The temporal aspect comes from the sequence of these daily records—today's closing price is influenced by yesterday's closing price, the trading volume, market sentiment, and numerous other factors.

#### **Key Characteristics of Time-Series Prediction Systems**

**1. Temporal Dependence**

The most defining characteristic is that observations are not independent. In our NEPSE example:

```python
# This illustrates temporal dependence
# Today's close price depends on previous days' prices

import pandas as pd

# Load NEPSE data
nepse_data = pd.read_csv('nepse_data.csv')

# For a single stock, let's examine the relationship
stock_data = nepse_data[nepse_data['Symbol'] == 'ABL'].copy()

# Calculate the correlation between consecutive close prices
# This demonstrates temporal dependence
stock_data['Prev_Close'] = stock_data['Close'].shift(1)

# Calculate correlation between today's close and yesterday's close
correlation = stock_data['Close'].corr(stock_data['Prev_Close'])
print(f"Correlation between consecutive close prices: {correlation:.4f}")
```

**Output Explanation**:
```
Correlation between consecutive close prices: 0.9987
```

The extremely high correlation (close to 1.0) confirms that consecutive daily closing prices are highly dependent on each other. This is the essence of temporal dependence—if yesterday's price is 588, today's price is very likely to be close to 588, not suddenly jump to 300 or 900 without significant market events.

**2. Sequential Nature**

Time-series data must maintain its order. Shuffling the data destroys critical information:

```python
# Demonstrating why order matters in time-series

import numpy as np
import matplotlib.pyplot as plt

# Create a simple visualization showing the difference
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Original ordered data
axes[0].plot(stock_data['Close'].values[:50], marker='o', markersize=3)
axes[0].set_title('Original Ordered Time-Series\n(Shows Trends and Patterns)')
axes[0].set_xlabel('Time Index (Days)')
axes[0].set_ylabel('Close Price (NPR)')

# Shuffled data - temporal information destroyed
shuffled = stock_data['Close'].values[:50].copy()
np.random.shuffle(shuffled)
axes[1].plot(shuffled, marker='o', markersize=3, color='orange')
axes[1].set_title('Shuffled Time-Series\n(Trends and Patterns Destroyed)')
axes[1].set_xlabel('Random Index (Meaningless)')
axes[1].set_ylabel('Close Price (NPR)')

plt.tight_layout()
plt.show()
```

**Visual Explanation**:
The left plot shows the natural progression of stock prices over 50 days—you can observe trends, potential reversals, and the natural ebb and flow of the market. The right plot shows the same data shuffled randomly—the temporal patterns are completely destroyed, making any meaningful prediction impossible.

**3. Time-Based Patterns**

Time-series data often exhibits patterns that repeat or evolve over time:

- **Trends**: Long-term upward or downward movement
- **Seasonality**: Regular, predictable patterns (e.g., higher trading volumes during earnings season)
- **Cyclicality**: Longer-term fluctuations without fixed periods
- **Irregularity**: Random, unpredictable variations

```python
# Identifying basic patterns in NEPSE data

def analyze_time_series_patterns(data, symbol, column='Close'):
    """
    Analyze and identify basic time-series patterns in stock data.
    
    Parameters:
    -----------
    data : pd.DataFrame
        The complete NEPSE dataset
    symbol : str
        Stock symbol to analyze (e.g., 'ABL')
    column : str
        Column to analyze (default: 'Close')
    
    Returns:
    --------
    dict : Dictionary containing pattern analysis results
    """
    
    # Filter data for the specific stock
    stock = data[data['Symbol'] == symbol].copy()
    stock = stock.sort_values('S.No').reset_index(drop=True)
    
    # Basic statistics
    mean_price = stock[column].mean()
    std_price = stock[column].std()
    
    # Trend analysis: Compare first half mean to second half mean
    mid_point = len(stock) // 2
    first_half_mean = stock[column][:mid_point].mean()
    second_half_mean = stock[column][mid_point:].mean()
    
    trend_direction = "Upward" if second_half_mean > first_half_mean else "Downward"
    trend_magnitude = abs(second_half_mean - first_half_mean) / first_half_mean * 100
    
    # Volatility analysis using rolling standard deviation
    stock['Rolling_Std_20'] = stock[column].rolling(window=20).std()
    avg_volatility = stock['Rolling_Std_20'].mean()
    
    return {
        'symbol': symbol,
        'mean_price': mean_price,
        'std_price': std_price,
        'trend_direction': trend_direction,
        'trend_magnitude': trend_magnitude,
        'avg_volatility': avg_volatility,
        'data_points': len(stock)
    }

# Example usage
# analysis = analyze_time_series_patterns(nepse_data, 'ABL')
# print(f"Trend: {analysis['trend_direction']} by {analysis['trend_magnitude']:.2f}%")
```

**Code Explanation**:

1. **Function Purpose**: This function takes raw stock data and performs basic pattern analysis, which is a fundamental first step in any time-series prediction system.

2. **Parameter Breakdown**:
   - `data`: The full dataset containing all stocks
   - `symbol`: The specific stock we want to analyze (like 'ABL' for Agriculture Development Bank)
   - `column`: Which price metric to focus on (Open, High, Low, Close, etc.)

3. **Trend Analysis Logic**:
   - We split the data into two halves
   - Compare the average price in the first half vs. second half
   - If the second half average is higher, the trend is upward
   - The magnitude tells us how significant the trend is

4. **Rolling Standard Deviation**:
   - This measures how much prices typically deviate from the mean over a 20-day window
   - Higher values indicate more volatile periods
   - This is crucial for risk assessment in prediction systems

#### **Components of a Time-Series Prediction System**

A complete prediction system consists of several interconnected components:

```
┌─────────────────────────────────────────────────────────────────┐
│                    TIME-SERIES PREDICTION SYSTEM                 │
├─────────────────────────────────────────────────────────────────┤
│                                                                  │
│  ┌──────────────┐    ┌──────────────┐    ┌──────────────┐       │
│  │ Data         │───▶│ Feature      │───▶│ Model        │       │
│  │ Collection   │    │ Engineering  │    │ Development  │       │
│  └──────────────┘    └──────────────┘    └──────────────┘       │
│         │                   │                   │                │
│         ▼                   ▼                   ▼                │
│  ┌──────────────┐    ┌──────────────┐    ┌──────────────┐       │
│  │ Data         │    │ Feature      │    │ Model        │       │
│  │ Storage      │    │ Selection    │    │ Training     │       │
│  └──────────────┘    └──────────────┘    └──────────────┘       │
│         │                   │                   │                │
│         └───────────────────┼───────────────────┘                │
│                             ▼                                    │
│                    ┌──────────────┐                              │
│                    │ Model        │                              │
│                    │ Evaluation   │                              │
│                    └──────────────┘                              │
│                             │                                    │
│                             ▼                                    │
│                    ┌──────────────┐                              │
│                    │ Deployment   │                              │
│                    │ & Monitoring │                              │
│                    └──────────────┘                              │
│                                                                  │
└─────────────────────────────────────────────────────────────────┘
```

---

### **1.2 Real-World Applications and Use Cases**

Time-series prediction systems are ubiquitous in modern technology and business. Understanding the breadth of applications helps us appreciate the versatility of the techniques we'll learn.

#### **Financial Markets (Our Focus: NEPSE)**

Financial time-series prediction is perhaps the most well-known application. In the context of NEPSE:

```python
# Financial application example: Price prediction for NEPSE stocks

import pandas as pd
import numpy as np

class NEPSEPricePredictor:
    """
    A simplified demonstration of a price prediction framework for NEPSE stocks.
    This is a conceptual framework - actual implementation would use sophisticated
    machine learning models which we'll develop throughout this handbook.
    """
    
    def __init__(self, data_path):
        """
        Initialize the predictor with NEPSE data.
        
        Parameters:
        -----------
        data_path : str
            Path to the CSV file containing NEPSE data
        """
        self.data = pd.read_csv(data_path)
        self.prepare_data()
    
    def prepare_data(self):
        """
        Prepare and clean the NEPSE data for analysis.
        
        This method handles:
        - Date conversion (if date column exists)
        - Missing value identification
        - Data type standardization
        - Basic validation
        """
        # Display basic information about the dataset
        print("=" * 60)
        print("NEPSE Dataset Overview")
        print("=" * 60)
        print(f"Total records: {len(self.data)}")
        print(f"Unique stocks: {self.data['Symbol'].nunique()}")
        print(f"\nColumns available: {list(self.data.columns)}")
        
        # Check for missing values
        missing = self.data.isnull().sum()
        if missing.any():
            print("\n⚠️ Missing values detected:")
            print(missing[missing > 0])
        else:
            print("\n✓ No missing values detected")
        
        # Display sample data
        print("\n" + "=" * 60)
        print("Sample Data (First 5 Records)")
        print("=" * 60)
        print(self.data.head())
        
        # Basic statistics for numeric columns
        print("\n" + "=" * 60)
        print("Statistical Summary")
        print("=" * 60)
        numeric_cols = ['Open', 'High', 'Low', 'Close', 'VWAP', 'Vol', 'Turnover']
        print(self.data[numeric_cols].describe())
    
    def get_stock_history(self, symbol):
        """
        Retrieve historical data for a specific stock.
        
        Parameters:
        -----------
        symbol : str
            Stock symbol (e.g., 'ABL', 'ADBL', 'NABIL')
        
        Returns:
        --------
        pd.DataFrame : Historical data for the specified stock
        """
        stock_data = self.data[self.data['Symbol'] == symbol].copy()
        stock_data = stock_data.sort_values('S.No').reset_index(drop=True)
        
        if len(stock_data) == 0:
            print(f"⚠️ No data found for symbol: {symbol}")
            return None
        
        print(f"\n{'=' * 60}")
        print(f"Stock History for {symbol}")
        print(f"{'=' * 60}")
        print(f"Records available: {len(stock_data)}")
        print(f"Date range: Record {stock_data['S.No'].min()} to {stock_data['S.No'].max()}")
        print(f"Price range: NPR {stock_data['Low'].min():.2f} - {stock_data['High'].max():.2f}")
        
        return stock_data

# Example usage (when data is available):
# predictor = NEPSEPricePredictor('nepse_data.csv')
# abl_history = predictor.get_stock_history('ABL')
```

**Detailed Code Explanation**:

1. **Class Structure**: We define a class `NEPSEPricePredictor` that encapsulates all prediction-related functionality. This follows object-oriented design principles and makes the code modular and reusable.

2. **`__init__` Method**:
   - This is the constructor that runs when we create a new instance of the predictor
   - It loads the CSV data using pandas
   - Calls `prepare_data()` to ensure data is ready for analysis
   - The `self` keyword refers to the current instance, allowing us to store data that persists across method calls

3. **`prepare_data` Method**:
   - **Purpose**: Ensures data quality before any analysis begins
   - **Information Display**: Shows total records, unique stocks, and available columns
   - **Missing Value Check**: Uses `isnull().sum()` to count null values per column
   - **Sample Data**: Displays first 5 records to give a quick visual verification
   - **Statistical Summary**: Provides count, mean, std, min, quartiles, and max for numeric columns

4. **`get_stock_history` Method**:
   - Filters the entire dataset to get records for a single stock symbol
   - Sorts by `S.No` (serial number) to maintain temporal order
   - Provides summary statistics for the specific stock
   - Returns the filtered DataFrame for further analysis

**Understanding the NEPSE Data Columns**:

| Column | Description | Example Value |
|--------|-------------|---------------|
| `S.No` | Serial number (acts as time index) | 1, 2, 3... |
| `Symbol` | Stock ticker symbol | ABL, ADBL, NABIL |
| `Conf.` | Confirmation status | Standard |
| `Open` | Opening price for the day (NPR) | 580.00 |
| `High` | Highest price during the day (NPR) | 590.00 |
| `Low` | Lowest price during the day (NPR) | 575.00 |
| `Close` | Closing price for the day (NPR) | 588.00 |
| `LTP` | Last Traded Price | 588.00 |
| `VWAP` | Volume Weighted Average Price | 584.50 |
| `Vol` | Trading volume (number of shares) | 15230 |
| `Turnover` | Total value traded (NPR) | 8900000 |
| `52 Weeks High/Low` | 52-week high and low prices | 650.00/420.00 |

#### **Other Industry Applications**

While our focus is financial prediction, understanding other domains helps appreciate the universality of time-series concepts:

**1. Retail and E-Commerce**

```python
# Example: Sales forecasting structure (adaptable to NEPSE-like format)

def retail_forecasting_example():
    """
    Demonstrates how time-series concepts apply to retail forecasting.
    The structure is similar to our NEPSE data but with retail-specific columns.
    """
    # Hypothetical retail data structure
    retail_data_structure = {
        'date': 'Trading day equivalent',
        'store_id': 'Stock symbol equivalent',
        'product_category': 'Sector equivalent',
        'units_sold': 'Volume equivalent',
        'revenue': 'Turnover equivalent',
        'avg_price': 'VWAP equivalent',
        'promotion_flag': 'External factor (like market events)'
    }
    
    print("Retail forecasting uses similar patterns:")
    print("- Daily/weekly seasonality (like trading day patterns)")
    print("- Holiday effects (like market holidays)")
    print("- Promotional impacts (like dividend announcements)")
    print("- Trend analysis (like market trends)")
    
    return retail_data_structure
```

**2. Energy Demand Forecasting**

```python
def energy_forecasting_structure():
    """
    Energy demand prediction structure comparison with NEPSE.
    """
    comparison = {
        'Aspect': ['Time Granularity', 'Prediction Target', 'Key Drivers', 'Seasonality'],
        'NEPSE': ['Daily', 'Stock Price', 'Market sentiment, Earnings', 'Quarterly patterns'],
        'Energy': ['Hourly', 'Electricity Demand', 'Temperature, Industry activity', 'Daily + Seasonal']
    }
    
    print("\n" + "=" * 70)
    print("Comparison: NEPSE vs Energy Forecasting")
    print("=" * 70)
    for i, aspect in enumerate(comparison['Aspect']):
        print(f"\n{aspect}:")
        print(f"  NEPSE:   {comparison['NEPSE'][i]}")
        print(f"  Energy:  {comparison['Energy'][i]}")
    
    return comparison
```

**3. Healthcare Monitoring**

```python
def healthcare_monitoring_example():
    """
    Patient vital signs monitoring as time-series (similar concepts to NEPSE).
    """
    # Structure for patient monitoring data
    patient_data_example = """
    Patient Monitoring Data Structure:
    ┌─────────────┬─────────────┬─────────────┬─────────────┐
    │ Timestamp   │ Heart Rate  │ Blood Press.│ Temperature │
    ├─────────────┼─────────────┼─────────────┼─────────────┤
    │ 10:00:00    │ 72          │ 120/80      │ 98.6        │
    │ 10:05:00    │ 75          │ 122/82      │ 98.7        │
    │ 10:10:00    │ 73          │ 119/79      │ 98.6        │
    └─────────────┴─────────────┴─────────────┴─────────────┘
    
    Similar to NEPSE where:
    - Timestamp ≈ S.No (time index)
    - Heart Rate ≈ Close (primary metric)
    - Blood Pressure ≈ High/Low range
    - Temperature ≈ VWAP (derived metric)
    
    Prediction Goals:
    - NEPSE: Will price go up or down?
    - Healthcare: Will patient's condition deteriorate?
    """
    print(patient_data_example)
```

---

### **1.3 Types of Prediction Problems**

Time-series prediction problems can be categorized into several types, each requiring different approaches and evaluation methods. Understanding these categories helps us choose the right techniques for our NEPSE prediction system.

#### **1.3.1 Regression Problems**

**Definition**: Predicting a continuous numerical value.

**NEPSE Example**: Predicting tomorrow's closing price.

```python
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

class NEPSERegressionPredictor:
    """
    Regression-based price prediction for NEPSE stocks.
    This demonstrates predicting continuous values (prices) from historical data.
    """
    
    def __init__(self, data):
        """
        Initialize with NEPSE data.
        
        Parameters:
        -----------
        data : pd.DataFrame
            NEPSE dataset containing historical stock information
        """
        self.data = data
        self.model = None
        self.feature_columns = None
    
    def create_features(self, symbol, lookback_days=5):
        """
        Create features for regression from historical data.
        
        This method transforms raw time-series data into a format suitable
        for machine learning by creating lag features and derived metrics.
        
        Parameters:
        -----------
        symbol : str
            Stock symbol to create features for
        lookback_days : int
            Number of previous days to use as features
        
        Returns:
        --------
        pd.DataFrame : Feature matrix with target variable
        """
        # Filter for specific stock and sort chronologically
        stock = self.data[self.data['Symbol'] == symbol].copy()
        stock = stock.sort_values('S.No').reset_index(drop=True)
        
        if len(stock) < lookback_days + 1:
            print(f"⚠️ Insufficient data for {symbol}. Need at least {lookback_days + 1} records.")
            return None
        
        # Create lag features: previous days' closing prices
        # These are the inputs (X) to our prediction model
        for i in range(1, lookback_days + 1):
            stock[f'Close_Lag_{i}'] = stock['Close'].shift(i)
        
        # Create derived features
        # These help the model understand patterns beyond raw prices
        
        # Price momentum: rate of change
        stock['Price_Momentum_1'] = stock['Close'] - stock['Close'].shift(1)
        stock['Price_Momentum_5'] = stock['Close'] - stock['Close'].shift(5)
        
        # Volatility: standard deviation over recent days
        stock['Volatility_5'] = stock['Close'].rolling(window=5).std()
        
        # Volume features
        stock['Volume_Lag_1'] = stock['Vol'].shift(1)
        stock['Volume_Change'] = stock['Vol'] / stock['Vol'].shift(1) - 1
        
        # Target: Next day's closing price
        # This is what we want to predict (y)
        stock['Target'] = stock['Close'].shift(-1)
        
        # Remove rows with NaN values (created by lag operations)
        stock = stock.dropna()
        
        # Define feature columns
        self.feature_columns = [col for col in stock.columns 
                                if 'Lag' in col or 'Momentum' in col 
                                or 'Volatility' in col or 'Change' in col]
        
        return stock
    
    def train(self, symbol, test_size=0.2):
        """
        Train a linear regression model for price prediction.
        
        Parameters:
        -----------
        symbol : str
            Stock symbol to train model for
        test_size : float
            Proportion of data to use for testing (0.0 to 1.0)
        
        Returns:
        --------
        dict : Training results including metrics
        """
        # Prepare features
        feature_data = self.create_features(symbol)
        
        if feature_data is None or len(feature_data) < 10:
            return {'error': 'Insufficient data for training'}
        
        # Split features (X) and target (y)
        X = feature_data[self.feature_columns]
        y = feature_data['Target']
        
        # Time-series split: use earlier data for training, later for testing
        # IMPORTANT: We don't use random splitting for time-series!
        split_idx = int(len(X) * (1 - test_size))
        X_train, X_test = X[:split_idx], X[split_idx:]
        y_train, y_test = y[:split_idx], y[split_idx:]
        
        # Initialize and train model
        self.model = LinearRegression()
        self.model.fit(X_train, y_train)
        
        # Make predictions
        y_pred = self.model.predict(X_test)
        
        # Calculate evaluation metrics
        metrics = {
            'MAE': mean_absolute_error(y_test, y_pred),
            'RMSE': np.sqrt(mean_squared_error(y_test, y_pred)),
            'R2': r2_score(y_test, y_pred),
            'train_size': len(X_train),
            'test_size': len(X_test)
        }
        
        # Store predictions for analysis
        self.predictions = pd.DataFrame({
            'Actual': y_test,
            'Predicted': y_pred,
            'Error': y_test - y_pred,
            'Abs_Error': np.abs(y_test - y_pred)
        })
        
        return metrics
    
    def analyze_predictions(self):
        """
        Analyze the prediction errors to understand model performance.
        """
        if self.predictions is None:
            print("⚠️ No predictions available. Run train() first.")
            return
        
        print("\n" + "=" * 60)
        print("Prediction Analysis")
        print("=" * 60)
        
        # Basic error statistics
        print("\n📊 Error Statistics:")
        print(f"   Mean Error:      {self.predictions['Error'].mean():.2f} NPR")
        print(f"   Std Dev Error:   {self.predictions['Error'].std():.2f} NPR")
        print(f"   Mean Abs Error:  {self.predictions['Abs_Error'].mean():.2f} NPR")
        
        # Percentage of predictions within different error thresholds
        actual = self.predictions['Actual']
        pct_error = (self.predictions['Abs_Error'] / actual) * 100
        
        print("\n📈 Prediction Accuracy:")
        print(f"   Within 1% error:  {(pct_error < 1).sum() / len(pct_error) * 100:.1f}%")
        print(f"   Within 2% error:  {(pct_error < 2).sum() / len(pct_error) * 100:.1f}%")
        print(f"   Within 5% error:  {(pct_error < 5).sum() / len(pct_error) * 100:.1f}%")

# Example usage:
# predictor = NEPSERegressionPredictor(nepse_data)
# metrics = predictor.train('ABL')
# print(f"R² Score: {metrics['R2']:.4f}")
# predictor.analyze_predictions()
```

**Detailed Code Walkthrough**:

1. **Feature Creation (`create_features` method)**:
   - **Lag Features**: We create `Close_Lag_1`, `Close_Lag_2`, etc., which represent the closing prices from 1, 2, 3... days ago. These are crucial because stock prices exhibit momentum—yesterday's price heavily influences today's.
   - **Momentum Features**: The difference between current price and prices from 1 or 5 days ago captures the direction and strength of price movement.
   - **Volatility Feature**: Rolling standard deviation over 5 days measures how unstable the price has been recently—high volatility often precedes significant moves.
   - **Volume Features**: Trading volume is a leading indicator; unusual volume often signals impending price moves.

2. **Training Process (`train` method)**:
   - **Time-Series Split**: Unlike typical ML problems where we shuffle data randomly, time-series requires chronological splitting. We train on past data to predict future data.
   - **Linear Regression**: The simplest model—we'll explore more sophisticated models in later chapters.
   - **Metrics Calculation**: MAE (Mean Absolute Error) gives average error in NPR; RMSE penalizes large errors more; R² measures how well the model explains variance.

3. **Error Analysis (`analyze_predictions` method)**:
   - Shows how close predictions are to actual values
   - Percentage-based thresholds are intuitive for financial data

#### **1.3.2 Classification Problems**

**Definition**: Predicting a categorical outcome.

**NEPSE Example**: Predicting whether the price will go UP or DOWN tomorrow.

```python
class NEPSEClassificationPredictor:
    """
    Classification-based direction prediction for NEPSE stocks.
    Instead of predicting exact price, we predict if price will rise or fall.
    """
    
    def __init__(self, data):
        self.data = data
        self.model = None
        self.feature_columns = None
    
    def create_classification_features(self, symbol, lookback_days=5):
        """
        Create features and binary target for classification.
        
        The target is:
        - 1 if tomorrow's close > today's close (price goes UP)
        - 0 if tomorrow's close <= today's close (price goes DOWN)
        
        Parameters:
        -----------
        symbol : str
            Stock symbol to analyze
        lookback_days : int
            Number of previous days to use as features
        
        Returns:
        --------
        pd.DataFrame : Feature matrix with binary target
        """
        stock = self.data[self.data['Symbol'] == symbol].copy()
        stock = stock.sort_values('S.No').reset_index(drop=True)
        
        # Create lag features (same as regression)
        for i in range(1, lookback_days + 1):
            stock[f'Close_Lag_{i}'] = stock['Close'].shift(i)
        
        # Price-based features
        stock['Price_Momentum_1'] = stock['Close'].pct_change()
        stock['Price_Range'] = (stock['High'] - stock['Low']) / stock['Close']
        
        # VWAP deviation: how far is close from VWAP?
        stock['VWAP_Deviation'] = (stock['Close'] - stock['VWAP']) / stock['VWAP']
        
        # Volume features
        stock['Volume_SMA_5'] = stock['Vol'].rolling(5).mean()
        stock['Volume_Ratio'] = stock['Vol'] / stock['Volume_SMA_5']
        
        # Relative position in 52-week range
        stock['Position_52W'] = (stock['Close'] - stock['52 Weeks Low']) / \
                                (stock['52 Weeks High'] - stock['52 Weeks Low'])
        
        # Create binary target: 1 if price goes UP, 0 if DOWN
        stock['Target'] = (stock['Close'].shift(-1) > stock['Close']).astype(int)
        
        # Clean data
        stock = stock.dropna()
        
        self.feature_columns = [col for col in stock.columns 
                                if 'Lag' in col or 'Momentum' in col 
                                or 'Range' in col or 'Deviation' in col
                                or 'Ratio' in col or 'Position' in col]
        
        return stock
    
    def train_classifier(self, symbol, test_size=0.2):
        """
        Train a classifier to predict price direction.
        
        Returns:
        --------
        dict : Training results including classification metrics
        """
        from sklearn.linear_model import LogisticRegression
        from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
        from sklearn.metrics import confusion_matrix, classification_report
        
        feature_data = self.create_classification_features(symbol)
        
        if feature_data is None or len(feature_data) < 10:
            return {'error': 'Insufficient data'}
        
        X = feature_data[self.feature_columns]
        y = feature_data['Target']
        
        # Chronological split
        split_idx = int(len(X) * (1 - test_size))
        X_train, X_test = X[:split_idx], X[split_idx:]
        y_train, y_test = y[:split_idx], y[split_idx:]
        
        # Train logistic regression classifier
        self.model = LogisticRegression(random_state=42, max_iter=1000)
        self.model.fit(X_train, y_train)
        
        # Predict
        y_pred = self.model.predict(X_test)
        y_prob = self.model.predict_proba(X_test)[:, 1]  # Probability of UP
        
        # Metrics
        metrics = {
            'accuracy': accuracy_score(y_test, y_pred),
            'precision': precision_score(y_test, y_pred),
            'recall': recall_score(y_test, y_pred),
            'f1_score': f1_score(y_test, y_pred)
        }
        
        # Confusion matrix interpretation
        cm = confusion_matrix(y_test, y_pred)
        
        print("\n" + "=" * 60)
        print(f"Classification Results for {symbol}")
        print("=" * 60)
        
        print("\n📊 Confusion Matrix:")
        print("                 Predicted")
        print("                 DOWN    UP")
        print(f"Actual DOWN    {cm[0][0]:5d}  {cm[0][1]:5d}")
        print(f"       UP      {cm[1][0]:5d}  {cm[1][1]:5d}")
        
        print("\n📈 Classification Metrics:")
        print(f"   Accuracy:  {metrics['accuracy']:.4f}")
        print(f"   Precision: {metrics['precision']:.4f}")
        print(f"   Recall:    {metrics['recall']:.4f}")
        print(f"   F1 Score:  {metrics['f1_score']:.4f}")
        
        # Feature importance
        feature_importance = pd.DataFrame({
            'Feature': self.feature_columns,
            'Coefficient': self.model.coef_[0]
        }).sort_values('Coefficient', key=abs, ascending=False)
        
        print("\n🔍 Top 5 Important Features:")
        print(feature_importance.head())
        
        return metrics

# Example usage:
# classifier = NEPSEClassificationPredictor(nepse_data)
# metrics = classifier.train_classifier('ABL')
```

**Understanding Classification vs. Regression**:

| Aspect | Regression | Classification |
|--------|------------|----------------|
| **Output** | Continuous value (e.g., 588.50) | Discrete category (UP/DOWN) |
| **NEPSE Goal** | Predict exact price | Predict direction |
| **Metric** | MAE, RMSE, R² | Accuracy, Precision, Recall |
| **Use Case** | Valuation, target setting | Trading signals, decisions |
| **Difficulty** | Harder (exact values) | Easier (direction only) |

**Interpreting Classification Metrics**:

- **Accuracy**: Overall percentage of correct predictions
  - Formula: `(TP + TN) / (TP + TN + FP + FN)`
  - In NEPSE context: How often do we correctly predict UP or DOWN?

- **Precision**: Of all UP predictions, how many were correct?
  - Formula: `TP / (TP + FP)`
  - In NEPSE context: When we say price will go UP, how often are we right?

- **Recall**: Of all actual UP movements, how many did we catch?
  - Formula: `TP / (TP + FN)`
  - In NEPSE context: Of all days the price actually went UP, how many did we predict correctly?

- **F1 Score**: Harmonic mean of Precision and Recall
  - Formula: `2 × (Precision × Recall) / (Precision + Recall)`
  - Useful when you need a balance between precision and recall

#### **1.3.3 Multi-Step Forecasting**

**Definition**: Predicting multiple future time points.

**NEPSE Example**: Predicting closing prices for the next 5 trading days.

```python
class NEPSEMultiStepForecaster:
    """
    Multi-step forecasting for NEPSE stocks.
    Predicts multiple future values instead of just one.
    """
    
    def __init__(self, data):
        self.data = data
        self.models = {}  # Store separate model for each horizon
    
    def create_multi_step_features(self, symbol, forecast_horizon=5):
        """
        Create features and multiple targets for multi-step forecasting.
        
        Parameters:
        -----------
        symbol : str
            Stock symbol
        forecast_horizon : int
            Number of future days to predict
        
        Returns:
        --------
        pd.DataFrame : Features with multiple target columns
        """
        stock = self.data[self.data['Symbol'] == symbol].copy()
        stock = stock.sort_values('S.No').reset_index(drop=True)
        
        # Create lag features
        for i in range(1, 11):  # Use 10 days of history
            stock[f'Close_Lag_{i}'] = stock['Close'].shift(i)
        
        # Create targets for each future step
        for h in range(1, forecast_horizon + 1):
            stock[f'Target_Day_{h}'] = stock['Close'].shift(-h)
        
        stock = stock.dropna()
        
        return stock
    
    def train_recursive(self, symbol, forecast_horizon=5):
        """
        Train recursive multi-step forecasting model.
        
        Recursive approach:
        1. Predict tomorrow's price
        2. Use that prediction to predict day after tomorrow
        3. Continue for all forecast horizons
        
        This is simpler but errors can accumulate.
        """
        from sklearn.linear_model import LinearRegression
        
        feature_data = self.create_multi_step_features(symbol, forecast_horizon)
        
        if len(feature_data) < 20:
            return {'error': 'Insufficient data'}
        
        # Features are the lag columns
        feature_cols = [col for col in feature_data.columns if 'Lag' in col]
        
        # Train a single model to predict next day
        # Then use recursively for further days
        X = feature_data[feature_cols]
        y = feature_data['Target_Day_1']
        
        split_idx = int(len(X) * 0.8)
        X_train, X_test = X[:split_idx], X[split_idx:]
        y_train, y_test = y[:split_idx], y[split_idx:]
        
        model = LinearRegression()
        model.fit(X_train, y_train)
        
        # Recursive forecasting
        predictions = []
        current_features = X_test.iloc[0:1].copy()
        
        for h in range(forecast_horizon):
            pred = model.predict(current_features)[0]
            predictions.append(pred)
            
            # Update features for next prediction
            # Shift all lag features and insert new prediction
            for i in range(len(feature_cols) - 1, 0, -1):
                current_features[feature_cols[i]] = current_features[feature_cols[i-1]].values
            current_features[feature_cols[0]] = pred
        
        return predictions
    
    def train_direct(self, symbol, forecast_horizon=5):
        """
        Train direct multi-step forecasting model.
        
        Direct approach:
        - Train a separate model for each forecast horizon
        - More accurate but requires more models
        
        Parameters:
        -----------
        symbol : str
            Stock symbol
        forecast_horizon : int
            Number of future days to predict
        
        Returns:
        --------
        dict : Predictions and models for each horizon
        """
        from sklearn.linear_model import LinearRegression
        
        feature_data = self.create_multi_step_features(symbol, forecast_horizon)
        
        if len(feature_data) < 20:
            return {'error': 'Insufficient data'}
        
        feature_cols = [col for col in feature_data.columns if 'Lag' in col]
        X = feature_data[feature_cols]
        
        split_idx = int(len(X) * 0.8)
        X_train, X_test = X[:split_idx], X[split_idx:]
        
        results = {'predictions': [], 'models': {}}
        
        for h in range(1, forecast_horizon + 1):
            target_col = f'Target_Day_{h}'
            y = feature_data[target_col]
            y_train, y_test = y[:split_idx], y[split_idx:]
            
            # Train separate model for this horizon
            model = LinearRegression()
            model.fit(X_train, y_train)
            
            # Store model
            self.models[h] = model
            results['models'][h] = model
            
            # Get prediction
            pred = model.predict(X_test.iloc[0:1])[0]
            results['predictions'].append(pred)
        
        return results

# Example usage:
# forecaster = NEPSEMultiStepForecaster(nepse_data)
# predictions = forecaster.train_direct('ABL', forecast_horizon=5)
# print("5-Day Forecast:", predictions['predictions'])
```

**Comparison of Multi-Step Approaches**:

| Approach | Description | Pros | Cons |
|----------|-------------|------|------|
| **Recursive** | Predict one step, use as input for next | Simple, single model | Errors accumulate |
| **Direct** | Separate model for each horizon | More accurate | More complex, more models |
| **Direct-Recursive** | Hybrid of both | Balanced accuracy | Implementation complexity |
| **Multiple Output** | Model outputs all predictions at once | Captures correlations | Limited model support |

---

### **1.4 The Machine Learning Pipeline Overview**

A machine learning pipeline for time-series prediction is a systematic sequence of steps that transform raw data into actionable predictions. Understanding this pipeline is crucial for building robust systems.

#### **The Complete Pipeline Architecture**

```
┌─────────────────────────────────────────────────────────────────────────┐
│                     MACHINE LEARNING PIPELINE                           │
├─────────────────────────────────────────────────────────────────────────┤
│                                                                         │
│  ┌─────────────┐   ┌─────────────┐   ┌─────────────┐   ┌─────────────┐ │
│  │   DATA      │   │   DATA      │   │   FEATURE   │   │   MODEL     │ │
│  │ COLLECTION  │──▶│   CLEANING  │──▶│ ENGINEERING │──▶│  SELECTION  │ │
│  └─────────────┘   └─────────────┘   └─────────────┘   └─────────────┘ │
│         │                 │                 │                 │         │
│         ▼                 ▼                 ▼                 ▼         │
│  ┌─────────────┐   ┌─────────────┐   ┌─────────────┐   ┌─────────────┐ │
│  │ - APIs      │   │ - Missing   │   │ - Lags      │   │ - Linear    │ │
│  │ - Files     │   │   values    │   │ - Rolling   │   │ - Tree      │ │
│  │ - Database  │   │ - Outliers  │   │ - Domain    │   │ - Neural    │ │
│  │ - Scraping  │   │ - Formats   │   │ - Time      │   │ - Ensemble  │ │
│  └─────────────┘   └─────────────┘   └─────────────┘   └─────────────┘ │
│                                                                         │
│  ┌─────────────┐   ┌─────────────┐   ┌─────────────┐   ┌─────────────┐ │
│  │   TRAINING  │   │  EVALUATION │   │  DEPLOYMENT │   │ MONITORING  │ │
│  │             │◀──│             │◀──│             │◀──│             │ │
│  └─────────────┘   └─────────────┘   └─────────────┘   └─────────────┘ │
│         │                 │                 │                 │         │
│         ▼                 ▼                 ▼                 ▼         │
│  ┌─────────────┐   ┌─────────────┐   ┌─────────────┐   ┌─────────────┐ │
│  │ - Split     │   │ - Metrics   │   │ - API       │   │ - Drift     │ │
│  │ - Validate  │   │ - Backtest  │   │ - Container │   │ - Retrain   │ │
│  │ - Tune      │   │ - Compare   │   │ - Scale     │   │ - Alert     │ │
│  │ - Save      │   │ - Report    │   │ - Version   │   │ - Log       │ │
│  └─────────────┘   └─────────────┘   └─────────────┘   └─────────────┘ │
│                                                                         │
└─────────────────────────────────────────────────────────────────────────┘
```

#### **Pipeline Implementation for NEPSE**

```python
import pandas as pd
import numpy as np
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error
import warnings
warnings.filterwarnings('ignore')

class NEPSEPipeline:
    """
    Complete ML pipeline implementation for NEPSE price prediction.
    This class demonstrates the end-to-end workflow from data loading
    to model evaluation.
    """
    
    def __init__(self, data_path=None, data=None):
        """
        Initialize the pipeline.
        
        Parameters:
        -----------
        data_path : str, optional
            Path to CSV file containing NEPSE data
        data : pd.DataFrame, optional
            Pre-loaded NEPSE data
        """
        if data is not None:
            self.raw_data = data.copy()
        elif data_path is not None:
            self.raw_data = pd.read_csv(data_path)
        else:
            raise ValueError("Either data_path or data must be provided")
        
        # Pipeline state
        self.cleaned_data = None
        self.feature_data = None
        self.model = None
        self.scaler = None
        self.pipeline_results = {}
    
    # ============================================================
    # STEP 1: DATA COLLECTION (Already done - data is loaded)
    # ============================================================
    
    def validate_raw_data(self):
        """
        Validate that the raw data meets minimum requirements.
        
        Returns:
        --------
        dict : Validation results
        """
        print("\n" + "=" * 60)
        print("STEP 1: DATA VALIDATION")
        print("=" * 60)
        
        required_columns = ['S.No', 'Symbol', 'Open', 'High', 'Low', 'Close']
        
        validation = {
            'total_records': len(self.raw_data),
            'unique_symbols': self.raw_data['Symbol'].nunique(),
            'columns_present': True,
            'missing_columns': [],
            'data_types_ok': True
        }
        
        # Check required columns
        for col in required_columns:
            if col not in self.raw_data.columns:
                validation['columns_present'] = False
                validation['missing_columns'].append(col)
        
        # Report
        print(f"✓ Total records: {validation['total_records']:,}")
        print(f"✓ Unique stocks: {validation['unique_symbols']}")
        
        if validation['columns_present']:
            print("✓ All required columns present")
        else:
            print(f"⚠ Missing columns: {validation['missing_columns']}")
        
        return validation
    
    # ============================================================
    # STEP 2: DATA CLEANING
    # ============================================================
    
    def clean_data(self, symbol=None):
        """
        Clean the NEPSE data.
        
        This step handles:
        - Missing value detection and treatment
        - Duplicate removal
        - Data type conversion
        - Basic outlier detection
        
        Parameters:
        -----------
        symbol : str, optional
            Specific stock to clean. If None, cleans all data.
        
        Returns:
        --------
        pd.DataFrame : Cleaned data
        """
        print("\n" + "=" * 60)
        print("STEP 2: DATA CLEANING")
        print("=" * 60)
        
        # Start with raw data
        data = self.raw_data.copy()
        
        # Filter for specific symbol if provided
        if symbol:
            data = data[data['Symbol'] == symbol].copy()
            print(f"Processing symbol: {symbol}")
        
        # Initial data shape
        initial_shape = data.shape
        print(f"\nInitial data shape: {initial_shape}")
        
        # 2.1: Check for duplicates
        duplicates = data.duplicated().sum()
        if duplicates > 0:
            data = data.drop_duplicates()
            print(f"✓ Removed {duplicates} duplicate rows")
        else:
            print("✓ No duplicates found")
        
        # 2.2: Handle missing values
        missing = data.isnull().sum()
        total_missing = missing.sum()
        
        if total_missing > 0:
            print(f"\n⚠ Total missing values: {total_missing}")
            print("Missing by column:")
            for col, count in missing[missing > 0].items():
                print(f"   {col}: {count}")
            
            # Strategy: Forward fill for price columns, drop others
            price_cols = ['Open', 'High', 'Low', 'Close', 'VWAP']
            for col in price_cols:
                if col in data.columns and data[col].isnull().any():
                    data[col] = data[col].fillna(method='ffill')
                    print(f"   Filled {col} with forward fill")
        else:
            print("✓ No missing values found")
        
        # 2.3: Sort by serial number (time order)
        data = data.sort_values('S.No').reset_index(drop=True)
        print("✓ Data sorted chronologically")
        
        # 2.4: Validate price consistency
        # High should be >= Low, High >= Open, High >= Close, etc.
        invalid_prices = (
            (data['High'] < data['Low']) |
            (data['High'] < data['Open']) |
            (data['High'] < data['Close']) |
            (data['Low'] > data['Open']) |
            (data['Low'] > data['Close'])
        ).sum()
        
        if invalid_prices > 0:
            print(f"⚠ Found {invalid_prices} records with inconsistent prices")
        else:
            print("✓ All price records are consistent")
        
        # Final shape
        final_shape = data.shape
        print(f"\nFinal data shape: {final_shape}")
        print(f"Records removed: {initial_shape[0] - final_shape[0]}")
        
        self.cleaned_data = data
        return data
    
    # ============================================================
    # STEP 3: FEATURE ENGINEERING
    # ============================================================
    
    def engineer_features(self, lookback=10):
        """
        Create features for the prediction model.
        
        Feature engineering transforms raw data into meaningful inputs
        that capture patterns relevant for prediction.
        
        Parameters:
        -----------
        lookback : int
            Number of historical days to create features from
        
        Returns:
        --------
        pd.DataFrame : Data with engineered features
        """
        print("\n" + "=" * 60)
        print("STEP 3: FEATURE ENGINEERING")
        print("=" * 60)
        
        if self.cleaned_data is None:
            print("⚠ No cleaned data. Run clean_data() first.")
            return None
        
        data = self.cleaned_data.copy()
        
        # ========================================
        # 3.1: Lag Features
        # ========================================
        # These capture the autoregressive nature of time series
        print("\nCreating lag features...")
        
        for i in range(1, lookback + 1):
            data[f'Close_Lag_{i}'] = data['Close'].shift(i)
            # Also lag volume
            data[f'Vol_Lag_{i}'] = data['Vol'].shift(i)
        
        print(f"  ✓ Created {lookback * 2} lag features")
        
        # ========================================
        # 3.2: Rolling Window Features
        # ========================================
        # These capture trends and volatility over windows
        print("\nCreating rolling window features...")
        
        windows = [5, 10, 20]
        for window in windows:
            # Rolling mean (trend)
            data[f'Close_MA_{window}'] = data['Close'].rolling(window).mean()
            
            # Rolling std (volatility)
            data[f'Close_STD_{window}'] = data['Close'].rolling(window).std()
            
            # Rolling volume mean
            data[f'Vol_MA_{window}'] = data['Vol'].rolling(window).mean()
        
        print(f"  ✓ Created {len(windows) * 3} rolling features")
        
        # ========================================
        # 3.3: Price-Based Features
        # ========================================
        print("\nCreating price-based features...")
        
        # Daily return
        data['Return'] = data['Close'].pct_change()
        
        # Price range (volatility indicator)
        data['Price_Range'] = data['High'] - data['Low']
        data['Price_Range_Pct'] = data['Price_Range'] / data['Close']
        
        # Gap (open vs previous close)
        data['Gap'] = data['Open'] - data['Close'].shift(1)
        data['Gap_Pct'] = data['Gap'] / data['Close'].shift(1)
        
        # VWAP deviation
        data['VWAP_Deviation'] = (data['Close'] - data['VWAP']) / data['VWAP']
        
        print("  ✓ Created 6 price-based features")
        
        # ========================================
        # 3.4: Momentum Features
        # ========================================
        print("\nCreating momentum features...")
        
        # Rate of change at different periods
        for period in [1, 5, 10]:
            data[f'ROC_{period}'] = (
                (data['Close'] - data['Close'].shift(period)) / 
                data['Close'].shift(period) * 100
            )
        
        # Price position in 52-week range
        data['Position_52W'] = (
            (data['Close'] - data['52 Weeks Low']) /
            (data['52 Weeks High'] - data['52 Weeks Low'])
        )
        
        print("  ✓ Created 4 momentum features")
        
        # ========================================
        # 3.5: Target Variable
        # ========================================
        print("\nCreating target variable...")
        
        # Next day's close (for regression)
        data['Target_Close'] = data['Close'].shift(-1)
        
        # Direction (for classification)
        data['Target_Direction'] = (data['Close'].shift(-1) > data['Close']).astype(int)
        
        print("  ✓ Created 2 target variables")
        
        # ========================================
        # 3.6: Clean and Report
        # ========================================
        initial_rows = len(data)
        data = data.dropna()
        final_rows = len(data)
        
        feature_columns = [col for col in data.columns 
                          if col not in ['S.No', 'Symbol', 'Conf.', 'LTP', 
                                         'Close - LTP', 'Close - LTP %',
                                         'Prev. Close', 'Trans.', 'Diff',
                                         'Diff %', 'Range %', 'VWAP %',
                                         'Target_Close', 'Target_Direction']]
        
        print("\n" + "-" * 40)
        print("FEATURE ENGINEERING SUMMARY")
        print("-" * 40)
        print(f"Total features created: {len(feature_columns)}")
        print(f"Rows removed (NaN): {initial_rows - final_rows}")
        print(f"Final dataset shape: {data.shape}")
        
        self.feature_data = data
        self.feature_columns = feature_columns
        
        return data
    
    # ============================================================
    # STEP 4: DATA SPLITTING
    # ============================================================
    
    def split_data(self, test_size=0.2, val_size=0.1):
        """
        Split data into train, validation, and test sets.
        
        CRITICAL: For time-series, we must preserve chronological order!
        We cannot randomly shuffle data like typical ML problems.
        
        Parameters:
        -----------
        test_size : float
            Proportion of data for testing
        val_size : float
            Proportion of training data for validation
        
        Returns:
        --------
        dict : Dictionary containing X_train, X_val, X_test, y_train, y_val, y_test
        """
        print("\n" + "=" * 60)
        print("STEP 4: DATA SPLITTING")
        print("=" * 60)
        
        if self.feature_data is None:
            print("⚠ No feature data. Run engineer_features() first.")
            return None
        
        data = self.feature_data.copy()
        
        # Features and target
        X = data[self.feature_columns]
        y = data['Target_Close']
        
        # Calculate split indices
        n = len(X)
        test_idx = int(n * (1 - test_size))
        train_idx = int(test_idx * (1 - val_size))
        
        # Split into train/val/test
        X_train = X[:train_idx]
        X_val = X[train_idx:test_idx]
        X_test = X[test_idx:]
        
        y_train = y[:train_idx]
        y_val = y[train_idx:test_idx]
        y_test = y[test_idx:]
        
        print(f"\nData splits:")
        print(f"  Training:   {len(X_train)} samples ({len(X_train)/n*100:.1f}%)")
        print(f"  Validation: {len(X_val)} samples ({len(X_val)/n*100:.1f}%)")
        print(f"  Test:       {len(X_test)} samples ({len(X_test)/n*100:.1f}%)")
        
        print(f"\n⚠ IMPORTANT: Data is NOT shuffled (time-series requirement)")
        print(f"   Training uses earliest {train_idx} records")
        print(f"   Validation uses records {train_idx} to {test_idx}")
        print(f"   Test uses records {test_idx} onwards")
        
        # Store splits
        self.splits = {
            'X_train': X_train, 'X_val': X_val, 'X_test': X_test,
            'y_train': y_train, 'y_val': y_val, 'y_test': y_test
        }
        
        return self.splits
    
    # ============================================================
    # STEP 5: FEATURE SCALING
    # ============================================================
    
    def scale_features(self):
        """
        Scale features to standardize their ranges.
        
        Feature scaling is important because:
        - Features have different scales (Close ~500, Vol ~10000)
        - Many ML algorithms work better with standardized features
        - Gradient descent converges faster with scaled features
        
        Returns:
        --------
        dict : Scaled train, validation, and test sets
        """
        print("\n" + "=" * 60)
        print("STEP 5: FEATURE SCALING")
        print("=" * 60)
        
        if self.splits is None:
            print("⚠ No data splits. Run split_data() first.")
            return None
        
        # Initialize scaler
        self.scaler = StandardScaler()
        
        # Fit scaler on training data ONLY
        # This prevents data leakage from validation/test sets
        X_train_scaled = self.scaler.fit_transform(self.splits['X_train'])
        
        # Transform validation and test using training statistics
        X_val_scaled = self.scaler.transform(self.splits['X_val'])
        X_test_scaled = self.scaler.transform(self.splits['X_test'])
        
        print("✓ Features scaled using StandardScaler")
        print("  (fit on training data, applied to all sets)")
        
        # Show example of scaling effect
        example_feature = self.feature_columns[0]
        example_idx = list(self.feature_columns).index(example_feature)
        
        original_mean = self.splits['X_train'][example_feature].mean()
        original_std = self.splits['X_train'][example_feature].std()
        
        print(f"\nExample: {example_feature}")
        print(f"  Original mean: {original_mean:.2f}")
        print(f"  Original std:  {original_std:.2f}")
        print(f"  Scaled mean:   {X_train_scaled[:, example_idx].mean():.4f}")
        print(f"  Scaled std:    {X_train_scaled[:, example_idx].std():.4f}")
        
        # Update splits
        self.splits['X_train_scaled'] = X_train_scaled
        self.splits['X_val_scaled'] = X_val_scaled
        self.splits['X_test_scaled'] = X_test_scaled
        
        return self.splits
    
    # ============================================================
    # STEP 6: MODEL TRAINING
    # ============================================================
    
    def train_model(self, model_type='linear'):
        """
        Train a prediction model.
        
        Parameters:
        -----------
        model_type : str
            Type of model to train ('linear', 'rf', 'xgboost')
        
        Returns:
        --------
        model : Trained model
        """
        print("\n" + "=" * 60)
        print("STEP 6: MODEL TRAINING")
        print("=" * 60)
        
        if 'X_train_scaled' not in self.splits:
            print("⚠ No scaled data. Run scale_features() first.")
            return None
        
        from sklearn.linear_model import Ridge
        from sklearn.ensemble import RandomForestRegressor
        
        # Select model
        if model_type == 'linear':
            self.model = Ridge(alpha=1.0)
            print("\nUsing Ridge Regression (L2 regularization)")
        elif model_type == 'rf':
            self.model = RandomForestRegressor(n_estimators=100, random_state=42)
            print("\nUsing Random Forest Regressor")
        else:
            raise ValueError(f"Unknown model type: {model_type}")
        
        # Train on training data
        X_train = self.splits['X_train_scaled']
        y_train = self.splits['y_train']
        
        self.model.fit(X_train, y_train)
        print(f"✓ Model trained on {len(X_train)} samples")
        
        # Validate on validation set
        X_val = self.splits['X_val_scaled']
        y_val = self.splits['y_val']
        
        val_predictions = self.model.predict(X_val)
        val_mae = mean_absolute_error(y_val, val_predictions)
        
        print(f"\nValidation MAE: {val_mae:.4f}")
        
        return self.model
    
    # ============================================================
    # STEP 7: MODEL EVALUATION
    # ============================================================
    
    def evaluate_model(self):
        """
        Evaluate the trained model on test data.
        
        This step assesses how well the model generalizes to unseen data.
        
        Returns:
        --------
        dict : Evaluation metrics
        """
        print("\n" + "=" * 60)
        print("STEP 7: MODEL EVALUATION")
        print("=" * 60)
        
        if self.model is None:
            print("⚠ No trained model. Run train_model() first.")
            return None
        
        from sklearn.metrics import (mean_squared_error, r2_score, 
                                     mean_absolute_percentage_error)
        
        # Test predictions
        X_test = self.splits['X_test_scaled']
        y_test = self.splits['y_test']
        
        predictions = self.model.predict(X_test)
        
        # Calculate metrics
        metrics = {
            'MAE': mean_absolute_error(y_test, predictions),
            'RMSE': np.sqrt(mean_squared_error(y_test, predictions)),
            'R2': r2_score(y_test, predictions),
            'MAPE': mean_absolute_percentage_error(y_test, predictions) * 100
        }
        
        print("\n📊 Test Set Performance:")
        print(f"   MAE:  {metrics['MAE']:.4f} NPR")
        print(f"   RMSE: {metrics['RMSE']:.4f} NPR")
        print(f"   R²:   {metrics['R2']:.4f}")
        print(f"   MAPE: {metrics['MAPE']:.2f}%")
        
        # Direction accuracy
        actual_direction = (y_test.values[1:] > y_test.values[:-1])
        pred_direction = (predictions[1:] > predictions[:-1])
        direction_accuracy = (actual_direction == pred_direction).mean()
        
        print(f"\n📈 Direction Accuracy: {direction_accuracy*100:.1f}%")
        print(f"   (How often the model correctly predicts UP/DOWN)")
        
        # Store results
        self.pipeline_results['metrics'] = metrics
        self.pipeline_results['predictions'] = predictions
        self.pipeline_results['actual'] = y_test.values
        
        return metrics
    
    # ============================================================
    # RUN COMPLETE PIPELINE
    # ============================================================
    
    def run_full_pipeline(self, symbol, model_type='linear'):
        """
        Execute the complete ML pipeline.
        
        This method runs all steps in sequence, demonstrating
        the end-to-end workflow.
        
        Parameters:
        -----------
        symbol : str
            Stock symbol to analyze
        model_type : str
            Type of model to train
        
        Returns:
        --------
        dict : Complete pipeline results
        """
        print("\n" + "=" * 60)
        print("RUNNING COMPLETE ML PIPELINE")
        print(f"Symbol: {symbol}")
        print("=" * 60)
        
        # Step 1: Validate
        self.validate_raw_data()
        
        # Step 2: Clean
        self.clean_data(symbol)
        
        # Step 3: Engineer features
        self.engineer_features(lookback=10)
        
        # Step 4: Split
        self.split_data()
        
        # Step 5: Scale
        self.scale_features()
        
        # Step 6: Train
        self.train_model(model_type)
        
        # Step 7: Evaluate
        metrics = self.evaluate_model()
        
        print("\n" + "=" * 60)
        print("PIPELINE COMPLETE")
        print("=" * 60)
        
        return self.pipeline_results


# ============================================================
# EXAMPLE USAGE
# ============================================================

# Create sample NEPSE data for demonstration
def create_sample_nepse_data(n_records=500, n_symbols=5):
    """
    Create synthetic NEPSE-like data for demonstration.
    
    This generates realistic-looking stock data that follows
    typical market patterns.
    """
    np.random.seed(42)
    
    symbols = ['ABL', 'ADBL', 'NABIL', 'NICA', 'SCB']
    
    all_data = []
    
    for symbol in symbols[:n_symbols]:
        # Generate price series with random walk + trend
        n = n_records
        
        # Starting price varies by symbol
        base_prices = {'ABL': 500, 'ADBL': 600, 'NABIL': 800, 'NICA': 400, 'SCB': 700}
        start_price = base_prices.get(symbol, 500)
        
        # Random walk with slight upward drift
        returns = np.random.normal(0.0005, 0.02, n)
        prices = start_price * np.cumprod(1 + returns)
        
        for i in range(n):
            close = prices[i]
            # High/Low around close
            high = close * (1 + abs(np.random.normal(0, 0.01)))
            low = close * (1 - abs(np.random.normal(0, 0.01)))
            open_price = close * (1 + np.random.normal(0, 0.005))
            
            volume = int(np.random.lognormal(9, 0.5))
            vwap = (high + low + close) / 3 * (1 + np.random.normal(0, 0.002))
            
            record = {
                'S.No': i + 1,
                'Symbol': symbol,
                'Conf.': 'Standard',
                'Open': round(open_price, 2),
                'High': round(high, 2),
                'Low': round(low, 2),
                'Close': round(close, 2),
                'LTP': round(close, 2),
                'Close - LTP': 0.0,
                'Close - LTP %': 0.0,
                'VWAP': round(vwap, 2),
                'Vol': volume,
                'Prev. Close': round(prices[i-1] if i > 0 else start_price, 2),
                'Turnover': int(volume * close),
                'Trans.': int(volume * 0.1),
                'Diff': round(close - prices[i-1] if i > 0 else 0, 2),
                'Range': round(high - low, 2),
                'Diff %': round((close - prices[i-1]) / prices[i-1] * 100 if i > 0 else 0, 2),
                'Range %': round((high - low) / close * 100, 2),
                'VWAP %': round((close - vwap) / vwap * 100, 2),
                '52 Weeks High': round(close * 1.3, 2),
                '52 Weeks Low': round(close * 0.7, 2)
            }
            all_data.append(record)
    
    df = pd.DataFrame(all_data)
    return df


# Generate sample data and demonstrate pipeline
sample_data = create_sample_nepse_data(n_records=500, n_symbols=3)
print("Sample NEPSE Data Generated:")
print(sample_data.head(10))

# Run pipeline
pipeline = NEPSEPipeline(data=sample_data)
results = pipeline.run_full_pipeline(symbol='ABL', model_type='linear')
```

**Complete Pipeline Explanation**:

This comprehensive pipeline demonstrates every critical step in building a time-series prediction system. Let me explain each component in detail:

**Step 1 - Data Validation**:
- Verifies that all required columns exist
- Checks for basic data integrity
- Reports on data volume and uniqueness

**Step 2 - Data Cleaning**:
- **Duplicate Removal**: Ensures each record is unique
- **Missing Value Handling**: Uses forward-fill for prices (appropriate for time-series)
- **Price Validation**: Ensures High ≥ Low, High ≥ Open, etc.
- **Chronological Sorting**: Critical for time-series analysis

**Step 3 - Feature Engineering**:
- **Lag Features**: Capture autocorrelation (relationship with past values)
- **Rolling Windows**: Capture trends and volatility over time
- **Price-Based Features**: Capture intra-day patterns
- **Momentum Features**: Capture rate of change and relative position

**Step 4 - Data Splitting**:
- Uses chronological split (not random!)
- Creates separate train/validation/test sets
- Explains why shuffling is wrong for time-series

**Step 5 - Feature Scaling**:
- Standardizes features to mean=0, std=1
- Fits scaler on training data only (prevents leakage)
- Applies same transformation to all sets

**Step 6 - Model Training**:
- Selects appropriate model
- Trains on training data
- Validates on validation data

**Step 7 - Model Evaluation**:
- Tests on held-out test data
- Calculates multiple metrics
- Reports direction accuracy (important for trading)

---
